# Satellite Image Processing in Python
This notebook demonstrates how to calculate NDVI, NDWI, and EVI indices from satellite images using Python. Follow the step-by-step guide below.

## Step 1: Install Required Libraries
Ensure you have the required Python libraries installed:
- `rasterio`: For handling GeoTIFF files
- `numpy`: For numerical operations
- `matplotlib`: For visualization

Run the following command in your terminal if not already installed:
```bash
pip install rasterio numpy matplotlib
```

## Step 2: Load the Satellite Image
Use `rasterio` to open the GeoTIFF image and explore its metadata. Replace `image_path` with the path to your satellite image file.

In [ ]:
import rasterio
import numpy as np

# Load the satellite image file (GeoTIFF format)
image_path = "path_to_your_satellite_image.tif"  # Replace with the actual file path

# Open the file using rasterio
dataset = rasterio.open(image_path)

# Print basic information about the image
print(f"Number of bands: {dataset.count}")
print(f"Image dimensions: {dataset.width}x{dataset.height}")
print(f"Coordinate Reference System: {dataset.crs}")

## Step 3: Extract Individual Bands
Identify the bands required for calculating NDVI, NDWI, and EVI. This example assumes Landsat 8:
- **Red Band**: Band 4
- **NIR Band**: Band 5
- **Green Band**: Band 3
- **Blue Band**: Band 2

In [ ]:
# Read the necessary bands
red = dataset.read(4)   # Band 4: Red
nir = dataset.read(5)   # Band 5: NIR
green = dataset.read(3) # Band 3: Green
blue = dataset.read(2)  # Band 2: Blue

## Step 4: Compute NDVI, NDWI, and EVI
Use the formulas below to calculate each index:
- **NDVI**: `(NIR - Red) / (NIR + Red)`
- **NDWI**: `(Green - NIR) / (Green + NIR)`
- **EVI**: `G * (NIR - Red) / (NIR + C1 * Red - C2 * Blue + L)`
Where:
- `G = 2.5`, `C1 = 6.0`, `C2 = 7.5`, `L = 1.0`

In [ ]:
# Avoid division by zero using a small constant (e.g., 1e-10)
ndvi = (nir - red) / (nir + red + 1e-10)

ndwi = (green - nir) / (green + nir + 1e-10)

# EVI constants
G, C1, C2, L = 2.5, 6.0, 7.5, 1.0
evi = G * (nir - red) / (nir + C1 * red - C2 * blue + L + 1e-10)

## Step 5: Visualize the Results
Use `matplotlib` to plot NDVI, NDWI, and EVI values.

In [ ]:
import matplotlib.pyplot as plt

# Plot NDVI
plt.figure(figsize=(10, 8))
plt.title("NDVI")
plt.imshow(ndvi, cmap='RdYlGn')  # Red-Green colormap for vegetation
plt.colorbar()
plt.show()

# Plot NDWI
plt.figure(figsize=(10, 8))
plt.title("NDWI")
plt.imshow(ndwi, cmap='Blues')  # Blue colormap for water
plt.colorbar()
plt.show()

# Plot EVI
plt.figure(figsize=(10, 8))
plt.title("EVI")
plt.imshow(evi, cmap='YlGn')  # Yellow-Green colormap for vegetation
plt.colorbar()
plt.show()

## Step 6: Save the Indices as GeoTIFF
Save the computed indices for further use or sharing.

In [ ]:
from rasterio.transform import from_bounds

# Define metadata for output file
meta = dataset.meta
meta.update(driver='GTiff', dtype='float32', count=1)

# Save NDVI
with rasterio.open("ndvi_output.tif", 'w', **meta) as dst:
    dst.write(ndvi.astype(np.float32), 1)

# Save NDWI
with rasterio.open("ndwi_output.tif", 'w', **meta) as dst:
    dst.write(ndwi.astype(np.float32), 1)

# Save EVI
with rasterio.open("evi_output.tif", 'w', **meta) as dst:
    dst.write(evi.astype(np.float32), 1)